# Titanic
This notebook is my second attempt at building a classification model for the Titanic dataset ([here](https://github.com/iherman10/titanic/blob/main/titanic.ipynb) is my original). I'm building a new one after completing Chapter 3 of Hands-On Machine Learning, which went through the steps to build and evaluate classification models (using the MNIST dataset). 

I might reference code from Chapter 3 of Hands-On Machine Learning ([notebook](https://github.com/ageron/handson-ml2/blob/master/03_classification.ipynb)). 

# Frame the problem
This is a classification task, as we need to predict whether a passenger survived or not. 

## Measuring performance
There are several metrics we can use to measure the performance of a classification model. That being said, Kaggle ranks submissions for this dataset based on accuracy (% of correct predictions). Both _precision_ and _recall_ should be high in order to have a highly accurate model. We can stick with __accuracy__ for now. 

# Get the data
We have a couple files avilable: `train.csv` and `test.csv`. 

In [4]:
import pandas as pd 
train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
train.shape, test.shape

((891, 12), (418, 11))

# Explore the data

In [10]:
# Create copy to explore the data
explore = train.copy()

In [11]:
explore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Quick observations: 
- `PassengerID` is not useful 
- `Survived` is the target variable, should be stored separately in target/label array.  
- `Pclass` is really more of a categorical variable, can perform OneHot encoding on it, 
- `Age` has significant number of missing values which should be imputed. Also should be normalized/standardized. 
- `Ticket` is probably not so useful, unclear what the data really means there. 
- `Fare` should be normalized/standardized. 
- `Cabin` has too many missing values. This can be dropped. 
- `Embarked` can be OneHot encoded. 

In [12]:
explore.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200
